# Extractive QA with RoBERTa & Hugging Face

## 📋 Overview
This notebook implements an **Extractive Question Answering** system. Unlike generative models that "write" an answer, this system identifies the exact span of text (start and end indices) within a provided context that best answers a given question. It utilizes the `RoBERTa` architecture fine-tuned on the SQuAD 2.0 dataset.

---

## 🛠️ Workflow Steps

### 1. Model Selection & Setup
* **Model**: Uses `deepset/roberta-base-squad2`, a robust transformer model optimized specifically for understanding context and extracting answers.
* **Hugging Face Hub**: Demonstrates how to load models and tokenizers seamlessly using `AutoModelForQuestionAnswering`.

### 2. Manual Inference (Low-Level)
* **Tokenization**: Questions and contexts are paired and converted into input tensors.
* **Logit Analysis**: The model outputs `start_logits` and `end_logits`.
    * The code uses `torch.argmax` to find the most probable start and end positions of the answer within the context.
* **Decoding**: Converts the numerical token IDs back into human-readable text.



### 3. Pipeline Inference (High-Level)
* **Abstraction**: Implements the Hugging Face `pipeline` API, which encapsulates tokenization, forward pass, and decoding into a single line of code.
* **Efficiency**: This method is preferred for production as it handles all the heavy lifting (like sliding windows for long contexts) automatically.

### 4. Architecture Insight: BERT & Context
* The notebook includes a practical example of BERT's **Bidirectional** approach, allowing it to look at the words both to the left and the right of a token to understand its full meaning.



---

## 📚 Key Tech Stack
| Category | Tools |
| :--- | :--- |
| **Model Hub** | `Hugging Face Transformers` |
| **Model** | `RoBERTa (SQuAD 2.0)` |
| **Tensors** | `PyTorch (torch)` |
| **Pipeline** | `transformers.pipeline` |

In [2]:
import torch
import pandas as pd
from transformers import AutoModelForQuestionAnswering,AutoTokenizer,pipeline

In [6]:
QA_input=[{
'question':'How to learn NLP?','context':'NLP stands for Natural Language Processing.'
' It is a field of artificial intelligence that enables computers to understand and process human language. To learn NLP, you can start with online courses, tutorials, and books on the subject. It is also helpful to practice by working on projects and using NLP libraries such as NLTK, spaCy, and Hugging Face Transformers.'

},
{'question':"Explain Architecture of BERT?",'context':'BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based model that uses a bidirectional approach to understand the context of words in a sentence.'
' It consists of multiple layers of transformers, which are attention-based mechanisms that allow the model to focus on different parts of the input text. BERT is pre-trained on a large corpus of text and can be fine-tuned for various '
'NLP tasks such as question answering, sentiment analysis, and named entity recognition.'}]

In [3]:
#select from the hub of hugging face
model_name="deepset/roberta-base-squad2" 




In [ ]:
# Model 1
model=AutoModelForQuestionAnswering.from_pretrained(model_name) 
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [ ]:
inputs0=tokenizer(QA_input[0]['question'],QA_input[0]['context'],return_tensors='pt')

outputs0=model(**inputs0)

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [10]:
input1=tokenizer(QA_input[1]['question'],QA_input[1]['context'],return_tensors='pt')
outputs1=model(**input1)


tensor(5.8542, grad_fn=<SelectBackward0>)

In [22]:
answer_start_idx=torch.argmax(outputs0.start_logits)
answer_end_idx=torch.argmax(outputs0.end_logits)


answer_start_idx

tensor(43)

In [23]:
print(outputs0.start_logits[0][43])

tensor(5.8542, grad_fn=<SelectBackward0>)


In [24]:
answer_tokens=inputs0.input_ids[0,answer_start_idx:answer_end_idx+1]
answer=tokenizer.decode(answer_tokens)
print("question {} answer: {}".format(QA_input[0]['question'],answer))


question How to learn NLP? answer:  online courses, tutorials, and books on the subject


In [25]:
#now for the second question
answer_start_idx1=torch.argmax(outputs1.start_logits) 
answer_end_idx1=torch.argmax(outputs1.end_logits) 
answer_tokens1=input1.input_ids[0,answer_start_idx1:answer_end_idx1+1] 
answer1=tokenizer.decode(answer_tokens1)
print("question {} answer: {}".format(QA_input[1]['question'],answer1))


question Explain Architecture of BERT? answer:  multiple layers of transformers


In [27]:
#method 2 using pipeline
qa=pipeline('question-answering',model=model_name,tokenizer=model_name)
result=qa(question=QA_input[0]['question'],context=QA_input[0]['context']) 
print("question {} answer: {}".format(QA_input[0]['question'],result['answer'])) 


Device set to use cuda:0


question How to learn NLP? answer: online courses, tutorials, and books on the subject


In [28]:

result1=qa(question=QA_input[1]['question'],context=QA_input[1]['context']) 
print("question {} answer: {}".format(QA_input[1]['question'],result1['answer']))


question Explain Architecture of BERT? answer: multiple layers of transformers
